## Web scraping of RouteScanner

This code is a work in progress and not even close to ready.

In [1]:
import sys
sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310')
sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310\\lib\\site-packages')

### Try static stuf

In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
Limit = 1000
Date = "2022-10-11"
BASE_URL = 'https://www.routescanner.com/'
Origin = "NLRTM"
Destination = "BRSSZ"

url = f"https://www.routescanner.com/voyages?limit={Limit}&from={Origin}&fromType=locode&originsNearby=1&to={Destination}&toType=locode&destinationsNearby=1&departure={Date}&sort=emission_co2"
print(url)

https://www.routescanner.com/voyages?limit=1000&from=NLRTM&fromType=locode&originsNearby=1&to=BRSSZ&toType=locode&destinationsNearby=1&departure=2022-10-11&sort=emission_co2


In [4]:
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [5]:
page.json

<bound method Response.json of <Response [200]>>

In [6]:
soup

<!DOCTYPE html>
<html lang="en" translate="no" xmlns:fb="http://ogp.me/ns/fb#"><head><meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/><meta content="notranslate" name="google"/><meta charset="utf-8"><title>Routescanner – Make better container shipping decisions</title><meta content="Find the best door-to-door shipping routes by sea and inland. Compare options on lead time and CO2 emissions." name="description"/><meta content="routescanner, container shipping routes, maritime route, routeplanner, shortsea connections, deepsea connections, rail connections, barge connections" name="keywords"/><meta content="For the best container shipping route comparison, use Routescanner to find the best connection by deepsea, feeder, rail, barge and truck." itemprop="name"/><meta content="Routescanner – Make better container shipping decisions" property="og:title"/><meta content="Routescanner – Plan your door-to-door container shipping route" property="og:site_name"

In [7]:
dates = soup.find_all("div", {"class" : re.compile('voyagesSection*')})

In [8]:
dates

[]

In [9]:
results = soup.find(id="voyagesSection*")
print(results.prettify())

AttributeError: 'NoneType' object has no attribute 'prettify'

### Try some dynamic stuff

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import os
import itertools
import pandas as pd

https://scrapingant.com/blog/scrape-dynamic-website-with-python

In [11]:
os.getcwd()

'C:\\Users\\Ewout\\Documents\\GitHub\\shipping-data'

In [12]:
# Define origin and destination countries
origin = ["BR"]
destination = ["NL", "BE"]

# Make list with all combinations
od_list = list(itertools.product(origin, destination))
print(od_list)

# Create URL list to scrape
urls = [f"https://www.routescanner.com/services/direct-connections/results?destinationCountries={o}&originCountries={d}&search=advanced" for o, d in od_list]

[('BR', 'NL'), ('BR', 'BE')]


In [13]:
url1 = urls[0]

# Instantiate options
opts = Options()
# opts.add_argument(" — headless") # Uncomment if the headless version needed
opts.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"

# Set the location of the webdriver
chrome_driver = os.getcwd() + "/drivers/chromedriver.exe"

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)

# Load the HTML page
driver.get(url1)

# Accept cookies
driver.implicitly_wait(2)
driver.find_element(By.CLASS_NAME,"acceptButton__P2szu").click()

# Wait untill route data is loaded
elem = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'card__hoI9D')))
print("Page is ready!")

# Parse processed webpage with BeautifulSoup
soup = BeautifulSoup(driver.page_source)

C:\Users\Ewout\AppData\Local\Temp\ipykernel_6964\815965137.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)


Page is ready!


#### Extract data from HTML

In [15]:
# Create a list with all route cards
routes = soup.find_all("div", class_="card__hoI9D")
route = routes[1]

# Find data for single route
c = route.find("h6").text
o, d = route.find("small", class_="locodes__tdIbs").text.split(' - ')
s = route.find("p", class_="serviceCode__igooW").text
print(s)

NWC TO SAEC - STRING I


In [16]:
# Define dict keys / dataframe columns
keys = ["Origin", "Destination", "Company", "Duration", "Frequency"]
route_dict = dict.fromkeys(keys)

for route in routes:
    route_dict["Origin"], route_dict["Destination"] = route.find("small", class_="locodes__tdIbs").text.split(' - ')

# Create empty DataFrame
route_df = pd.DataFrame.from_dict(route_dict, orient="index", columns=keys)
route_df

ValueError: Shape of passed values is (5, 1), indices imply (5, 5)

In [17]:
route_dict

{'Origin': 'NLRTM',
 'Destination': 'BRFOR',
 'Company': None,
 'Duration': None,
 'Frequency': None}